# VICA Technical Assessment - Task 2

Name of candidate: Chan Choon Kong

## Import libraries

In [10]:
# !pip install pymongo
import pymongo
from pymongo import MongoClient
from collections import OrderedDict

## Making a connection with MongoClient

In [4]:
client = MongoClient('localhost', 27017)
db = client['test']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test')

## Create collection for data

In [6]:
db.create_collection("insuranceData")
db.list_collection_names()

[]
['insuranceData']


## Define validation rules

In [13]:
schema = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["insuree#", "isMarried", "hasKids", "insuredMonths", "termLifeInsurance", "healthInsurance", "eStatements", "monthlyPremium", "renewal"],
        "properties": {
            "insuree#": {
                "bsonType": "int",
                "description": "ID  of the insuree"
            },
            "gender": {
                "enum": ["M", "F"],
                "description": "M or F"
            },
            "is45OrOlder": {
                "bsonType": "bool",
                "description": "Is insuree >= 45 years old?"
            },
            "isMarried": {
                "bsonType": "bool",
                "description": "Is insuree married?"
            },
            "hasKids": {
                "bsonType": "bool",
                "description": "Does the insuree have kids?"
            },
            "insuredMonths": {
                "bsonType": "int",
                "description": "Months of active insurance"
            },
            "termLifeInsurance": {
                "bsonType": "object",
                "required": ["hasPolicy", "hasMultiplePolicies"],
                "properties": {
                    "hasPolicy": {
                        "bsonType": "bool",
                        "description": "Does insuree have term life policy?"
                    },
                    "hasMultiplePolicies": {
                        "bsonType": "bool",
                        "description": "Does insuree hold multiple term life policies?"
                    }
                }
            },
            "healthInsurance": {
                "bsonType": "object",
                "required": ["hasPolicy", "riders"],
                "properties": {
                    "hasPolicy": {
                        "bsonType": "bool",
                        "description": "Does insuree have health insurance?"
                    },
                    "riders": {
                        "bsonType": "array",
                        "items": {
                            "enum": [1, 2, 3, 4]
                        },
                        "description": "Does insuree have riders on this health policy?"
                    }
                }
            },
            "premiumFrequency": {
                "enum": [1, 3, 12],
                "description": "Premium due monthly, quarterly, annually"
            },
            "eStatements": {
                "bsonType": "bool",
                "description": "Opted in for e-statements and e-policies?"
            },
            "monthlyPremium": {
                "bsonType": "double",
                "description": "Premium amount monthly"
            },
            "totalPremium": {
                "bsonType": "double",
                "description": "Total premium amount"
            },
            "renewal": {
                "bsonType": "bool",
                "description": "Does insuree renew at next premium cycle?"
            }
        }
    }
}

cmd = OrderedDict([('collMod', 'insuranceData'), ('validator', schema)])
db.command(cmd)

{'ok': 1.0}